$\newcommand{\is}{\mathrel{\mathop:=}}$
$\newcommand{\range}{\mathop{ran}}$
$\newcommand{\setof}[1]{\left \{ #1 \right \}}$
$\newcommand{\card}[1]{\left | #1 \right |}$
$\newcommand{\tuple}[1]{\left \langle #1 \right \rangle}$
$\newcommand{\emptytuple}{\left \langle \right \rangle}$
$\newcommand{\tuplecat}{\cdot}$
$\newcommand{\stringcat}{\cdot}$
$\newcommand{\emptystring}{\varepsilon}$
$\newcommand{\String}[1]{\mathit{#1}}$
$\newcommand{\LeftEdgeSymbol}{\rtimes}$
$\newcommand{\RightEdgeSymbol}{\ltimes}$
$\newcommand{\LeftEdge}{\LeftEdgeSymbol}$
$\newcommand{\RightEdge}{\RightEdgeSymbol}$
$\newcommand{\mult}{\times}$
$\newcommand{\multisum}{\uplus}$
$\newcommand{\multimult}{\otimes}$
$\newcommand{\freqsymbol}{\mathrm{freq}}$
$\newcommand{\freq}[1]{\freqsymbol(#1)}$
$\newcommand{\prob}{P}$
$\newcommand{\counts}[2]{\card{#2}_{#1}}$
$\newcommand{\inv}[1]{#1^{-1}}$
$\newcommand{\Lex}{\mathit{Lex}}$
$\newcommand{\length}[1]{\left | #1 \right |}$
$\newcommand{\suc}{S}$
$\newcommand{\sprec}{<}$
$\newcommand{\Rcomp}[2]{#1 \circ #2}$
$\newcommand{\domsymbol}{\triangleleft}$
$\newcommand{\idom}{\domsymbol}$
$\newcommand{\pdom}{\domsymbol^+}$
$\newcommand{\rdom}{\domsymbol^*}$
$\newcommand{\indegree}[1]{\mathrm{in(#1)}}$
$\newcommand{\outdegree}[1]{\mathrm{out(#1)}}$
$\newcommand{\cupdot}{\cup\mkern-11.5mu\cdot\mkern5mu}$
$\newcommand{\mymatrix}[1]{\left ( \matrix{#1} \right )}$

do not read yet; work in progress

# Removing Stop Words

The problem of cluttered sets is a very real one.
This is due to **Zipf's law**, which was formulated by the linguist [George Kingsley Zipf](https://en.wikipedia.org/wiki/George_Kingsley_Zipf).

<div class=definition>
    Let $t$ be a text of natural language utterances, and $R$ a total ranking of words by their frequency in $t$.
    Then the frequency $f$ of a word in $t$ is inversely proprotional to its rank $r$ in $R$ ($f = \frac{1}{r}$).
</div>

Zipf's law tell us that the most common word in a text occurs

- two times as often as the second most common one,
- three times as often as the third most common one,
- four times as often as the fourth most common one,
- and so on.

So if the most common word occurs 100 times, the second occurs 50 times, the third one 33 times, the fourth one 25 times, and so on.
As in all statistical claims, there is a certain margin of error, so the second word may actually occur 57 times and the third one 26 times.
But overall the frequency distribution of a natural language will have a very peculiar shape, where a few words make up the majority of a text while many other words occur only once.

<div class=exercise>
Suppose you have a text with 20,000 words where the most frequent word occurs 1068 times.
Assuming a distribution that is exactly Zipfian, calculate the frequencies for rank 2, 3, 5, 15, 50, 100, 200, 500, and 1000.
Give a rough esitmate how many of the top ranks jointly make up at least 50% of the text.
</div>

Let us sharpen our terminology a bit so that we can express this idea more clearly.
We need to distinguish between the *word type*, which is the word in the abstract sense, and the *word token*, the concrete instances of the word.
For example, the sentence *John is obviously John* contains the word types *John*, *obviously*, and *is*.
But it contains two tokens of the type *John*, one token of the type *obviously*, and one token of the types *is*.
Note that with this new terminology, the set-of-words model maps a sequence of word tokens to a set of word types.

Returning to Zipf's law, we can now state its consequence in a less misleading manner: very few word types make up the majority of word tokens in a text, whereas the majority of word types only have one token each in the text.
If we draw a plot with the word types in a text in descending frequency along the $x$-axis and their number of tokens as the $y$-axis, this produces a curve that starts out high, drops off quickly, and then has a very long tail that approaches 0 but never reaches it.

In [ ]:
# import plotting library as plt
import matplotlib.pyplot as plt
import numpy as np

# define a function with Zipfian distribution
def f(x):
    return 1/x

# possible values for x: 1, 2, 3, 4, ..., 250
x = np.arange(1, 250)

# compute f(x) for every x
plt.plot(x, f(x))

# show the plotted diagram
plt.show()

Zipf's law is more than an interesting curiosity, it has noticeable repercussions in practice.
In fact, it is one of the primary reasons why language technology is still much less capable than humans despite the recent advances in machine learning.
Machine learning algorithms are designed to focus on the robust, very frequent aspects of a data set while ignoring the rare outliers.
But Zipf's law tells us that this is exactly the wrong approach for language.
Here is what you get if you take the first 250 words of *Moby Dick* and throw away any words that aren't among the 250 most common in the book.

In [ ]:
import urllib.request
import re
from collections import Counter

urllib.request.urlretrieve("http://www.gutenberg.org/files/2701/2701.txt", "mobydick.txt")
with open("mobydick.txt", "r") as text:
    # tokenize string into list of lowercase words
    text = re.split("[^\w]", text.read().lower())
    # compute most frequent word types
    most_frequent = [count[0] for count in Counter(text).most_common(250)]
    # find the beginning of Chapter 1 in the file
    for n in range(len(text)):
        if text[n] == "call" and\
        text[n+1] == "me" and\
        text[n+2] == "ishmael":
            start = n
            break
    try:
        # and now take the first 250 words and remove all infrequent ones
        filtered = [word for word in text[n:n+250]
                    if word in most_frequent]
        # and put the tokenized list back into a string
        print(" ".join(filtered))
    except:
        print("Couldn't find start")

Not too illuminating, is it?
With language, it is the outliers that are interesting rather than the very frequent stuff.
An algorithm that favors the excessively common over the rare is ill-suited for language.

Alright, let us return to our original set-of-words model and improve it by removing uninformative word types from the unigram set.
There are multiple ways we can do this, but first we must agree on what counts as an uninformative word type.
For the sake of simplicity, we will consider a word type uninformative iff it is a **stop word**, i.e. a word type with a very high token frequency.

Here is a list of English stop words (using a very lenient definition of word that also includes contractions):

> a, about, above, after, again, against, all, am, an, and, any, are, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can't, cannot, could, couldn't, did, didn't, do, does, doesn't, doing, don't, down, during, each, few, for, from, further, had, hadn't, has, hasn't, have, haven't, having, he, he'd, he'll, he's, her, here, here's, hers, herself, him, himself, his, how, how's, i, i'd, i'll, i'm, i've, if, in, into, is, isn't, it, it's, its, itself, let's, me, more, most, mustn't, my, myself, no, nor, not, of, off, on, once, only, or, other, ought, our, ours, ourselves, out, over, own, same, shan't, she, she'd, she'll, she's, should, shouldn't, so, some, such, than, that, that's, the, their, theirs, them, themselves, then, there, there's, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, very, was, wasn't, we, we'd, we'll, we're, we've, were, weren't, what, what's, when, when's, where, where's, which, while, who, who's, whom, why, why's, with, won't, would, wouldn't, you, you'd, you'll, you're, you've, your, yours, yourself, yourselves